In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook as tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

EDAR_add = '../input/prepare-edar-data/edar_denoised_data.gz'
round1_add = '../input/prepare-round1-data/tianchi_round1.gz'
round2_add = '../input/prepare-data/tianchi_round2.gz'
ptbxl_add = '../input/download-physionet-data/phycom_ptbxl_data'
physionet_add = '../input/prepare-physionet-competition-expand/phycom_data.gz'
data_size = [10628,24106,20036,21837,42354]
output_data_size = [30000,30000,30000,sum(data_size)-90000]
print('data_size: ',data_size)
print('output_data_size: ',output_data_size)
read_each_time = int(30000/4)
print('#lines each time read from a gz file: ',read_each_time)

EDAR_read_rows = [int(data_size[0]/sum(data_size)*output_data_size[0])] * 3
EDAR_read_rows += [data_size[0] - sum(EDAR_read_rows)]
print('EDAR_read_rows: ',EDAR_read_rows)

round1_read_rows = [int(data_size[1]/sum(data_size)*output_data_size[0])] * 3
round1_read_rows += [data_size[1] - sum(round1_read_rows)]
print('round1_read_rows: ',round1_read_rows)

round2_read_rows = [int(data_size[2]/sum(data_size)*output_data_size[0])] * 3
round2_read_rows += [data_size[2] - sum(round2_read_rows)]
print('round2_read_rows: ',round2_read_rows)

ptbxl_read_rows = [int(data_size[3]/sum(data_size)*output_data_size[0])] * 3
ptbxl_read_rows += [data_size[3] - sum(ptbxl_read_rows)]
print("ptbxl_read_rows: ",ptbxl_read_rows)

physionet_read_rows = [int(data_size[4]/sum(data_size)*output_data_size[0]) + 3] * 3
physionet_read_rows += [data_size[4] - sum(physionet_read_rows)]
print('physionet_read_rows: ',physionet_read_rows)

assert [EDAR_read_rows[item]+round1_read_rows[item]+round2_read_rows[item]+
ptbxl_read_rows[item]+physionet_read_rows[item] for item in range(4)] == output_data_size

EDAR_read_rows = [sum(EDAR_read_rows[:k+1]) for k in range(4)]
round1_read_rows = [sum(round1_read_rows[:k+1]) for k in range(4)]
round2_read_rows = [sum(round2_read_rows[:k+1]) for k in range(4)]
ptbxl_read_rows = [sum(ptbxl_read_rows[:k+1]) for k in range(4)]
physionet_read_rows = [sum(physionet_read_rows[:k+1]) for k in range(4)]
print('EDAR_read_rows: ',EDAR_read_rows)
print('round1_read_rows: ',round1_read_rows)
print('round2_read_rows: ',round2_read_rows)
print("ptbxl_read_rows: ",ptbxl_read_rows)
print('physionet_read_rows: ',physionet_read_rows)

ori_edar_label_to_123_lalel = '1 degree atrioventricular block\t1AVB\t12\t0\n2 degree atrioventricular block\t2AVB\t13\t1\n2 degree atrioventricular block(Type one)\t2AVB1\t14\t-1\n2 degree atrioventricular block(Type two)\t2AVB2\t15\t-1\n3 degree atrioventricular block\t3AVB\t16\t-1\natrial bigeminy\tABI\t17\t-1\nAxis left shift\tALS\t18\t-1\nabnormal Q wave\tAQW\t20\t71\nAxis right shift\tARS\t21\t75\natrioventricular block\tAVB\t22\t14\ncountercolockwise rotation\tCCR\t23\t-1\ncolockwise rotation\tCR\t24\t-1\nEarly repolarization of the ventricles\tERV\t25\t-1\nfQRS Wave\tFQRS\t26\t-1\nInterior differences conduction\tIDC\t27\t-1\nIntraventricular block\tIVB\t28\t26\njunctional escape beat\tJEB\t29\t38\nJ point shift\tJPS\t30\t-1\njunctional premature beat\tJPT\t31\t-1\nleft bundle branch block\tLBBB\t32\t47\nleft back bundle branch block\tLBBBB\t33\t-1\nleft front bundle branch block\tLFBBB\t34\t-1\nLong RR interval\tLRRI\t35\t-1\nleft ventricle hypertrophy\tLVH\t36\t49\nleft ventricle high voltage\tLVHV\t37\t-1\nlower voltage QRS in all lead\tLVQRSAL\t38\t51\nlower voltage QRS in chest lead\tLVQRSCL\t39\t-1\nlower voltage QRS in limb lead\tLVQRSLL\t40\t52\nmyocardial infarction\tMI\t41\t57\nmyocardial infraction in back wall\tMIBW\t42\t-1\nMyocardial infgraction in the front wall\tMIFW\t43\t-1\nMyocardial infraction in the lower wall\tMILW\t44\t-1\nMyocardial infraction in the side wall\tMISW\t45\t-1\nPR interval extension\tPRIE\t46\t70\nP wave Change\tPWC\t47\t-1\nQT interval extension\tQTIE\t48\t69\nright atrial hypertrophy\tRAH\t49\t74\nright atrial high voltage\tRAHV\t50\t-1\nright bundle branch block\tRBBB\t51\t76\nright ventricle hypertrophy\tRVH\t52\t77\nST drop down\tSTDD\t53\t84\nST extension\tSTE\t54\t85\nST-T Change\tSTTC\t55\t61\nST tilt up\tSTTU\t56\t85\nT wave Change\tTWC\t57\t90\nT wave opposite\tTWO\t58\t91\nU wave\tUW\t59\t93\nventricular bigeminy\tVB\t60\t94\nventricular escape beat\tVEB\t61\t96\nventricular fusion wave\tVFW\t62\t-1\nventricular premature beat\tVPB\t63\t87\nventricular preexcitation\tVPE\t64\t102\nventricular escape trigeminy\tVET\t65\t104\nWandering in the atrioventricalualr node\tWAVN\t66\t-1\nWPW\tWPW\t67\t106\natrial premature beats\tAPB\t19\t66\nSinus Bradycardia\tSB\t1\t81\nSinus Rhythm\tSR\t2\t82\nAtrial Fibrillation\tAFIB\t3\t8\nSinus Tachycardia\tST\t4\t83\nAtrial Flutter\tAF\t5\t10\nSinus Irregularity\tSI\t6\t-1\nSupraventricular Tachycardia\tSVT\t7\t88\nAtrial Tachycardia\tAT\t8\t13\nAtrioventricular  Node Reentrant Tachycardia\tAVNRT\t9\t-1\nAtrioventricular Reentrant Tachycardia\tAVRT\t10\t-1\nSinus Atrium to Atrial Wandering Rhythm\tSAAWR\t11\t-1\nSinus Arrhythmia\tSA\t0\t80'

f = open('ori_edar_label_to_123_lalel','w+')
f.write(ori_edar_label_to_123_lalel)
f.close()

edar_label_123_dict = ori_edar_label_to_123_lalel.split('\n')
edar_label_123_dict = {item.split('\t')[2]:item.split('\t')[3] for item in edar_label_123_dict}
print('edar_label_123_dict: ', edar_label_123_dict)

ori_round1_label_to_123_lalel = '窦性心律\t0\t82\nT波改变\t1\t-1\n窦性心动过缓\t2\t81\nST段改变\t3\t78\n正常ECG\t4\t-1\n左心室高电压\t5\t-1\nST-T改变\t6\t61\n窦性心动过速\t7\t83\n临界ECG\t8\t-1\nQRS低电压\t9\t51\n房性早搏\t10\t-1\n电轴左偏\t11\t46\n电轴右偏\t12\t75\n心房颤动\t13\t8\n异常ECG\t14\t-1\n室性早搏\t15\t68\n完全性右束支传导阻滞\t16\t24\n窦性心律不齐\t17\t80\n左心室肥大\t18\t49\n右束支传导阻滞\t19\t76\n一度房室传导阻滞\t20\t-1\n快心室率\t21\t-1\n不完全性右束支传导阻滞\t22\t33\n非特异性T波异常\t23\t-1\nQT间期延长\t24\t69\n左前分支传导阻滞\t25\t42\n非特异性ST段异常\t26\t-1\n差异性传导\t27\t-1\n非特异性ST段与T波异常\t28\t61\n起搏心律\t29\t63\n短PR间期\t30\t79\n下壁异常Q波\t31\t71\n快室率心房颤动\t32\t8\n逆钟向转位\t33\t-1\n室内差异性传导\t34\t-1\n早期复极化\t35\t27\n二联律\t36\t-1\n室上性早搏\t37\t87\n顺钟向转位\t38\t-1\n复极化异常\t39\t-1\n未下传的房性早搏\t40\t-1\n肺心病型\t41\t-1\n慢心室率\t42\t-1\n短串房性心动过速\t43\t13\n非特异性室内传导延迟\t44\t-1\n右心房扩大\t45\t74\n左束支传导阻滞\t46\t47\n前间壁R波递增不良\t47\t72\n右心室肥大\t48\t77\n房室传导延缓\t49\t65\n双分支传导阻滞\t50\t19\n非特异性室内传导阻滞\t51\t60\n肺型P波\t52\t-1\n完全性左束支传导阻滞\t53\t-1\n融合波\t54\t-1'

f = open('ori_round1_label_to_123_lalel','w+')
f.write(ori_round1_label_to_123_lalel)
f.close()

round1_label_123_dict = ori_round1_label_to_123_lalel.split('\n')
round1_label_123_dict = {item.split('\t')[1]:item.split('\t')[2] for item in round1_label_123_dict}
print('round1_label_123_dict: ', round1_label_123_dict)

ori_round2_label_to_123_lalel = 'QRS低电压\t0\t51\n电轴右偏\t1\t75\n起搏心律\t2\t63\nT波改变\t3\t-1\n电轴左偏\t4\t46\n心房颤动\t5\t8\n非特异性ST段异常\t6\t-1\n下壁异常Q波\t7\t71\n前间壁R波递增不良\t8\t72\nST段改变\t9\t78\n一度房室传导阻滞\t10\t-1\n左束支传导阻滞\t11\t47\n右束支传导阻滞\t12\t76\n完全性左束支传导阻滞\t13\t-1\n左前分支传导阻滞\t14\t42\n右心房扩大\t15\t74\n短PR间期\t16\t79\n左心室高电压\t17\t-1\n窦性心动过缓\t18\t81\n早期复极化\t19\t27\n窦性心律\t20\t82\n融合波\t21\t-1\nST-T改变\t22\t61\n非特异性ST段与T波异常\t23\t61\n快心室率\t24\t-1\n非特异性T波异常\t25\t-1\n室性早搏\t26\t68\n房性早搏\t27\t-1\n窦性心律不齐\t28\t80\n完全性右束支传导阻滞\t29\t24\n窦性心动过速\t30\t83\n不完全性右束支传导阻滞\t31\t33\n顺钟向转位\t32\t-1\n逆钟向转位\t33\t-1'

f = open('ori_round2_label_to_123_lalel','w+')
f.write(ori_round2_label_to_123_lalel)
f.close()

round2_label_123_dict = ori_round2_label_to_123_lalel.split('\n')
round2_label_123_dict = {item.split('\t')[1]:item.split('\t')[2] for item in round2_label_123_dict}
print('round2_label_123_dict: ', round2_label_123_dict)

import gzip
import base64
from random import shuffle
EDAR_f = gzip.open(EDAR_add,'rt')
round1_f = gzip.open(round1_add,'rt')
round2_f = gzip.open(round2_add,'rt')
ptbxl_f = gzip.open(ptbxl_add,'rt')
physionet_f = gzip.open(physionet_add,'rt')

time = 2
datas = []
num_new_label = 123


def load_data(f, read_rows, do_transfer, transfer_label_dict):
    for count, line in tqdm(enumerate(f)):
        if time > 0 and count < read_rows[time-1]:
            continue
        if count >= read_rows[time]: break
        name, data, label, feature = line.strip().split('\t')
        
        if do_transfer == 1:
            ori_labels = np.frombuffer(base64.b64decode(label), dtype=np.float16)
            new_labels = [0.] * num_new_label
            for ori_index in transfer_label_dict:
                new_index = transfer_label_dict[ori_index]
                if new_index == -1: continue
                ori_label = ori_labels[int(ori_index)]
                new_labels[int(new_index)] = ori_label
            new_labels = np.asarray(new_labels)
            label = base64.b64encode(new_labels.astype(np.float16)).decode("utf-8")
            
        datas.append([name, data, label, feature])
          
        assert len(np.frombuffer(base64.b64decode(label), dtype=np.float16)) == num_new_label
    
load_data(EDAR_f,EDAR_read_rows,1,edar_label_123_dict)
load_data(round1_f,round1_read_rows,1,round1_label_123_dict)
load_data(round2_f,round2_read_rows,1,round2_label_123_dict)
load_data(ptbxl_f,ptbxl_read_rows,0,{})
load_data(physionet_f,physionet_read_rows,0,{})

shuffle(datas)

count = 59999
for line in datas:
    out_f = open('ecg_data.part'+(str(count).zfill(6)),'w+')
    out_f.write('\t'.join(line))
    count += 1
count

/kaggle/input/prepare-physionet-competition-expand/__results__.html
/kaggle/input/prepare-physionet-competition-expand/label_dict
/kaggle/input/prepare-physionet-competition-expand/label_dict_added
/kaggle/input/prepare-physionet-competition-expand/__notebook__.ipynb
/kaggle/input/prepare-physionet-competition-expand/Dx_map.csv
/kaggle/input/prepare-physionet-competition-expand/custom.css
/kaggle/input/prepare-physionet-competition-expand/__output__.json
/kaggle/input/prepare-physionet-competition-expand/phycom_data.gz
/kaggle/input/prepare-edar-data/__results__.html
/kaggle/input/prepare-edar-data/__notebook__.ipynb
/kaggle/input/prepare-edar-data/edar_denoised_data.gz
/kaggle/input/prepare-edar-data/custom.css
/kaggle/input/prepare-edar-data/__output__.json
/kaggle/input/prepare-edar-data/edar_ori_data.gz
/kaggle/input/download-physionet-data/__results__.html
/kaggle/input/download-physionet-data/label_dict
/kaggle/input/download-physionet-data/__notebook__.ipynb
/kaggle/input/downlo

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:111: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


89999